In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tidy3d as td
from tidy3d.plugins import waveguide
from tidy3d.plugins.mode import ModeSolver

# --- 1. Material Definitions ---
# Using fixed indices for 1550nm for simplicity

wavelength =  1.55 # Central wavelength
points = 5
num_modes = 6
sweep_wavelength = np.linspace(1.5,1.6,points)  # Sweep on wavelengths
sweep_freq = td.C_0 / sweep_wavelength      # Sweep on Frequencies


mat_si = td.material_library["cSi"]["Li1993_293K"] # Material trace permitivity model for crystaline Silicon
mat_sio2 = td.material_library["SiO2"]["Horiba"]   # Material trace permitivity model for crystaline Silica


def get_guided_modes(pol_type, width=0.48, thickness=0.22, wavelength = np.array([1.55])):
    """
    Separates TE/TM using Symmetry and counts only guided modes.
    """
    # For a waveguide centered vertically:
    # TE (Ey dominant): Z-Symmetry is Even (1)
    # TM (Ez dominant): Z-Symmetry is Odd (-1)
    # Note: We assume the waveguide is centered at z = thickness/2
    sym_z = 1 if pol_type == 'TE' else -1

    # We ask for the material properties from the wavelength we are on

    # eps_complex_si =mat_si.eps_model(td.C_0/wavelength)
    #
    # n_si = np.sqrt(eps_complex_si).real
    # k_si = np.sqrt(eps_complex_si).imag
    #
    # eps_complex_sio2 =mat_sio2.eps_model(td.C_0/wavelength)
    #
    # n_sio2 = np.sqrt(eps_complex_sio2).real
    # k_sio2 = np.sqrt(eps_complex_sio2).imag


    # Define the Plugin
    wg_plugin = waveguide.RectangularDielectric(
        wavelength=wavelength,
        core_width=width,
        core_thickness=thickness,
        core_medium=mat_si,
        clad_medium=mat_sio2,
        box_medium=mat_sio2,
        mode_spec=td.ModeSpec(num_modes=num_modes, target_neff=3.475) # Ask for many
    )

    # Correct way to get the simulation and apply symmetry
    # We access the simulation through the internal mode_solver
    sim_base = wg_plugin.mode_solver.simulation
    sym_sim = sim_base.copy(update={"symmetry": (0, 0, sym_z)})


    # Re-initialize the solver with the symmetric simulation
    solver = ModeSolver(
        simulation=sym_sim,
        plane=wg_plugin.mode_solver.plane,
        mode_spec=wg_plugin.mode_solver.mode_spec,
        freqs=wg_plugin.mode_solver.freqs
    )

    data = solver.solve()

    # Filter for guided modes only (n_eff > n_cladding)
    # squeeze() handles the frequency dimension
    n_eff_return = []
    for freq_in in range(points):
        n_effs = data.n_eff.values.squeeze()[freq_in]
        guided_indices = np.where(n_effs > 1.458)[0]
        n_eff_return.append(n_effs[guided_indices])

    return wg_plugin, n_eff_return, data



# --- 2. Execution ---
w_test = 1.0  # Testing a multimode width


# Here we apply the defined function to obtain TE and TM modes separated and kept in te_neff as refractive effective index, and TE data kept on TE data
# Likewise with TM modes
_ ,te_neffs, te_data = get_guided_modes('TE', width=w_test, wavelength=sweep_wavelength)
strip_waveguide,tm_neffs, tm_data = get_guided_modes('TM', width=w_test, wavelength=sweep_wavelength)


# --- 3. Structure Visualization ---
# We plot the cross-section to verify materials and dimensions

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
strip_waveguide.plot_structures(x= 0, ax=ax)
# te_data.Ey.abs.isel(mode_index = 0).squeeze().plot(x = 'y', y = 'z')
plt.title("Transverse Structure: Si Strip with SiO2 BOX and Cladding")
plt.show()

# total_modes = len(te_neffs)+len(tm_neffs)
#
# # --- 5. Plotting Results for TE Light ---
# # Mode index 0 is typically the fundamental Quasi-TE mode
# fig1, axs = plt.subplots(len(te_neffs), 2, figsize=(14, 5))
#
# # Plot Ey field (dominant component for TE in this orientation)
# for modes in range(len(te_neffs)):
#     strip_waveguide.plot_field("Ey", "abs", mode_index = 0  , ax=axs[2*modes+1])  ## SI SE PUEDE DESDE LA DATA QUE DA EL SOLVER, POR LO QUE SI SE `PUEDE PLOTEAR POR SEPARADO.
#     axs[2*modes+1].set_title(f"Fundamental TE Mode (Ey) | n_eff: {float(te_data.n_eff.values[0][0]):.4f}")
#
#     # Plot E-field absolute intensity
#     strip_waveguide.plot_field("E", "abs", mode_index=0, ax=axs[2*modes+2])
#     axs[2*modes+2].set_title("Mode Intensity Profile")
#
#
# plt.tight_layout()
# plt.show()
#
# # Print out the effective indices for all solved modes
# for i, neff in enumerate(tm_data.n_eff.values[0]):
#     print(f"Mode {i}: n_eff = {neff:.4f}")



# --- 3. Results ---
print(f"--- Results for Width {w_test}um ---")
print(f"Found {len(te_neffs)} guided TE modes: {te_neffs}")
print(f"Found {len(tm_neffs)} guided TM modes: {tm_neffs}")

D:\GitHub\GDS_py_TDY_venv\.venv\Lib\site-packages\scipy\sparse\_construct.py:543: FutureWarning: Input has data type int64, but the output has been cast to float64.  In the future, the output data type will match the input. To avoid this warning, set the `dtype` parameter to `None` to have the output dtype match the input, or set it to the desired output data type.
Note: In Python 3.11, this warning can be generated by a call of scipy.sparse.diags(), but the code indicated in the warning message will refer to an internal call of scipy.sparse.diags_array(). If that happens, check your code for the use of diags().
  A = diags_array(diagonals, offsets=offsets, shape=shape, dtype=dtype)


22:18:54 Hora est. Pacífico, Sudamérica WARNING: Mode field at frequency index  
                                        0, mode index 4 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        0, mode index 5 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        1, mode index 4 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        1, mode index 5 does not decay at the   
                                        plane boundaries.                       

...

22:18:56 Hora est. Pacífico, Sudamérica WARNING: Mode field at frequency index  
                                        3, mode index 4 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        3, mode index 5 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        4, mode index 3 does not decay at the   
                                        plane boundaries.                       

                                        WARNING: Mode field at frequency index  
                                        4, mode index 4 does not decay at the   
                                        plane boundaries.                       

22:18:57 Hora est. Pacífico, Sudamérica WARNING: Mode field at frequency index  
                                        4, mode index 5 does not decay at the   
                                        plane boundaries.                       